In [ ]:
import torch

ovl = torch.hub.load('pyannote/pyannote-audio', 'ovl')

In [ ]:
test_file = {'uri': audio_id, 'audio': audio_files.get(audio_id)[0]}
ovl_scores = ovl(test_file)

In [ ]:
from pyannote.audio.utils.signal import Binarize
binarize = Binarize(offset=0.55, onset=0.55, log_scale=True, 
                    min_duration_off=0.1, min_duration_on=0.1)

# overlapped speech regions (as `pyannote.core.Timeline` instance)
overlap = binarize.apply(ovl_scores, dimension=1)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from pyannote.core import Segment, notebook

# only plot one minute (between t=120s and t=180s)
notebook.crop = Segment(20, 130)

# helper function to make visualization prettier
from pyannote.core import SlidingWindowFeature
plot_ready = lambda scores: SlidingWindowFeature(np.exp(scores.data[:, 1:]), scores.sliding_window)

nrows = 3
fig, ax = plt.subplots(nrows=nrows, ncols=1)
fig.set_figwidth(20)
fig.set_figheight(nrows * 2)

notebook.plot_annotation(annotation['bauzd'], ax=ax[0], time=False)
ax[0].text(notebook.crop.start + 0.5, 0.1, 'referência', fontsize=14)

notebook.plot_feature(plot_ready(ovl_scores), ax=ax[1], time=False)
ax[1].text(notebook.crop.start + 0.5, 0.2, 'pontuação\nfalas sobrepostas', fontsize=14)
ax[1].set_ylim(-0.1, 1.1)

# 7th row: OVL result
notebook.plot_timeline(overlap, ax=ax[2], time=False)
ax[2].text(notebook.crop.start + 0.5, 0.1, 'detecção de falas sobrepostas', fontsize=14)